In [1]:
import os
import random
from datetime import datetime

now = datetime.now()

# Folder name to save registration transforms
resultsFolder = "results/" + now.strftime("%Y_%m_%d_%H_%M_%S")
print("Results folder name:", resultsFolder)

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# random shuffle data
# random.shuffle(data)

# spilt dataset
trainData = data[:int(datasetLen * 0.7)]
testData = data[int(datasetLen * 0.7):]

"""Write transforms."""
from pathlib import Path


try:
    Path(f"./{resultsFolder}").mkdir(parents=True, exist_ok=True)
    print("Saving training data names.")
    file = open(f"./{resultsFolder}/trainData.txt", "w") 
    for i, name in enumerate(trainData):
        # print(f"\tSaving {name}. {i+1}/{len(trainData)}", end='\x1b[1K\r')
        # print(f"\tSaving {name}. {i+1}/{len(trainData)}")
        file.write(name+"\n")
    # print()
    file.close()

    print("Saving testData names.")
    file = open(f"./{resultsFolder}/testData.txt", "w") 
    for i, name in enumerate(testData):
        # print(f"\tSaving {name}. {i+1}/{len(testData)}", end='\x1b[1K\r')
        # print(f"\tSaving {name}. {i+1}/{len(testData)}")
        file.write(name+"\n")
    # print()
    file.close()
except:
    print("Can't save names.")


# Fork processes for each test image
testImageIdx = 0
# for i in range(1, len(testData)):
for i in range(1, 0):
    n = os.fork() 
    if n == 0: 
        # child
        testImageIdx = i
        break 

resultsFolder += f"/{testImageIdx:02d}"
try:
    Path(f"./{resultsFolder}").mkdir(parents=True, exist_ok=True)
except:
    print("Can't make the results directory.")


Results folder name: results/2020_11_20_01_25_02
Saving training data names.
Saving testData names.


In [2]:
# Tests
atlasesNums = [5]
Ps = [[3, 3, 3]]
Ns = [[5, 5, 5]]
lassoTols = [0.1, 0.01, 0.001, 0.0001]

inputData = []
for atlas in atlasesNums:
    for P in Ps:
        for N in Ns:
            for lassoTol in lassoTols:
                inputData.append((atlas, P, N, lassoTol))


In [ ]:
import SimpleITK as sitk
import numpy as np
from utils import *

# Set the number of threads
sitk.ImageRegistrationMethod().SetGlobalDefaultNumberOfThreads(1)

transforms = []
mseList = []

# find biggest P and N
transtransformParameterMapP = np.argmax(Ps, axis=0)[0]
N = np.argmax(Ns, axis=0)[0]
    
# Read fixed image
pathName = os.path.join(path, testData[testImageIdx])
fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10)


for i, f in enumerate(trainData[:1]): # for test
# for i, f in enumerate(trainData):
    # print(f'Registration of {f}. {i+1}/{len(trainData)}', end='\x1b[1K\r')
    print(f'Registration of {f}. {i+1}/{len(trainData)}')
    
    pathName = os.path.join(path, f)
                  
    # Histogram matching
    image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      timeStep = 0.04, 
                                                      numberOfIterations = 10),
                                   fixedImage)

    label = sitk.ReadImage(os.path.join(labelPath, f))

    # t = registration(fixedImage, image, label)
    # t = registration2(fixedImage, image, label)
    t = registrationElastix(fixedImage, image)

    # add transform to the list
    transforms.append(t)

    # add mse to the list
    mseList.append(mse3D(fixedImage, resampleImage(image, fixedImage, t)))
    
# sort mseList
mseListSorted = sorted(enumerate(mseList), key=lambda x: x[1])

###############################################################################
# For every test
###############################################################################
# for testIndex, (atlasesNum, P, N, lassoTol) in enumerate(inputData):
for testIndex, (atlasesNum, P, N, lassoTol) in enumerate(inputData[0:1]):
    # pick the desired number of atlases
    minx = miny = minz = 10000
    maxx = maxy = maxz = 0
    images = []
    labels = []
    
    # Read fixed image
    pathName = os.path.join(path, testData[testImageIdx])
    fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10)

    ###############################################################################
    # Read atlases and labels and tranform them
    ###############################################################################
    # for i in range(atlasesNum):
    for i in range(1):
        idx = mseListSorted[i][0]
        t = transforms[idx]
        f = trainData[idx]
        pathName = os.path.join(path, f)
        # print(f'Reading {f}. {i+1}/{atlasesNum}', end='\x1b[1K\r')
        print(f'Reading {f}. {i+1}/{atlasesNum}')

        # Histogram matching
        images.append(sitk.GetArrayFromImage(resampleImage(
                    sitk.HistogramMatching(
                        sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                           timeStep = 0.04, 
                                           numberOfIterations = 10),
                                        fixedImage), 
                      fixedImage, t)))
        images[i][images[i] < 0] = 0
        images[i] = np.asarray(images[i], dtype=np.uint16)

        label = sitk.ReadImage(os.path.join(labelPath, f))
        label = sitk.GetArrayFromImage(resampleLabels(label, fixedImage, t))
        labels.append(label)

        idxs = np.nonzero(label != 0)
        minx = min([minx, min(idxs[0])])
        maxx = max([maxx, max(idxs[0])])
        miny = min([miny, min(idxs[1])])
        maxy = max([maxy, max(idxs[1])])
        minz = min([minz, min(idxs[2])])
        maxz = max([maxz, max(idxs[2])])


    ###############################################################################
    # Calculate the desired shape
    ###############################################################################
    shape, copyShape, offset, length = calculateCropShape(images, P, N, minx, maxx, miny, maxy, minz, maxz)

    ###############################################################################
    # crop images and labels
    ###############################################################################
    for i in range(len(images)):
        images[i] = cropImage(images[i], shape, offset, length, copyShape, "uint16")
        labels[i] = cropImage(labels[i], shape, offset, length, copyShape, "uint8")

    fixedImage = sitk.GetArrayFromImage(fixedImage)
    fixedImage = cropImage(fixedImage, shape, offset, length, copyShape, "uint16")


    ###############################################################################
    # Make to the proper type
    ###############################################################################
    # print("Fixing data types.", end='\x1b[1K\r')
    print("Fixing data types.")
    images = np.array(images, order='C')
    labels = np.array(labels, order='C')
    P = np.array(P, dtype=np.int32)
    N = np.array(N, dtype=np.int32)

    ###############################################################################
    # Perform segmentation
    ###############################################################################
    from cGen import cGen
    print("Running segmentation.")
    '''
    segmentation = cGen.applySPBM(fixedImage, images, labels, 3, P, N, lassoTol=0.00001, 
                                  lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                                  xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)
    '''
    segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
                                  labels, 3, P, N, lassoTol=lassoTol, 
                                  lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                                  xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1,
                                  numThreads=1)
                                  # xmin=10, xmax=11, ymin=10, ymax=11, zmin=10, zmax=11)

    ###############################################################################
    # Save segmentation results
    ###############################################################################

    # print("Saving results.", end='\x1b[1K\r')
    print("Saving results.")
    try:
        saveSegmentation(segmentationSPBM,
                         os.path.join(path, testData[testImageIdx]), 
                         os.path.join(resultsFolder, testData[testImageIdx]) + f"SPBM_{testIndex}.mhd", 
                         copyShape, 
                         offset,
                         length,
                         verbose=False)
    except:
        print("Can't save SPBM result.")

    try:
        saveSegmentation(segmentationSRC,
                         os.path.join(path, testData[testImageIdx]), 
                         os.path.join(resultsFolder, testData[testImageIdx]) + f"SRC_{testIndex}.mhd", 
                         copyShape, 
                         offset,
                         length,
                         verbose=False)
    except:
        print("Can't save SRC result.")


    ###############################################################################
    # Save dice index
    ###############################################################################

    originalLabel = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(labelPath, testData[testImageIdx])))

    translatedSegmentationSPBM = translateToOriginal(segmentationSPBM, 
                                                     os.path.join(path, testData[testImageIdx]), 
                                                     copyShape, 
                                                     offset, 
                                                     length)

    diceSPBM = Dice(originalLabel, translatedSegmentationSPBM, 3)

    translatedSegmentationSRC = translateToOriginal(segmentationSRC, 
                                                    os.path.join(path, testData[testImageIdx]), 
                                                    copyShape, 
                                                    offset, 
                                                    length)

    diceSRC = Dice(originalLabel, translatedSegmentationSRC, 3)
    
    print(f"{testIndex}: Atlases: {atlasesNum}, P: {P}, N: {N}, lassoTol: {lassoTol}")
    print("SPBM:", diceSPBM)
    print("SRC:", diceSRC)

    try:
        diceFile = open(f"./{resultsFolder}/dice.txt", "a") 
        diceFile.write(f"{testIndex}: Atlases: {atlasesNum}, P: {P}, N: {N}, lassoTol: {lassoTol}"+"\n")
        diceFile.write("SPBM: " + str(diceSPBM) + "\n")
        diceFile.write("SRC: " + str(diceSRC) + "\n")
        diceFile.close()
    except:
        print("Can't append dice results.")

Registration of 9005132.nii.gz. 1/32
Reading 9005132.nii.gz. 1/5
Fixing data types.
Running segmentation.
X: 3 	Time: 4.4194419384002686
X: 4 	Time: 7.546820163726807
X: 5 	Time: 1.4726526737213135
X: 6 	Time: 1.4380345344543457
X: 7 	Time: 1.5412590503692627
X: 8 	Time: 1.7850379943847656
X: 9 	Time: 2.1380836963653564
X: 10 	Time: 1.9830358028411865
X: 11 	Time: 1.8670268058776855
X: 12 	Time: 1.3063085079193115
X: 13 	Time: 1.151723861694336
X: 14 	Time: 1.0739452838897705
X: 15 	Time: 1.1278355121612549
X: 16 	Time: 1.088543176651001
X: 17 	Time: 1.0340514183044434
X: 18 	Time: 1.4170880317687988
X: 19 	Time: 1.2643249034881592
X: 20 	Time: 1.277510166168213
X: 21 	Time: 1.6180553436279297
X: 22 	Time: 1.2638916969299316
X: 23 	Time: 1.3659982681274414
X: 24 	Time: 1.4349541664123535
X: 25 	Time: 1.4017858505249023
X: 26 	Time: 1.3677692413330078
X: 27 	Time: 1.456578254699707
X: 28 	Time: 1.754978895187378
X: 29 	Time: 1.1697616577148438
X: 30 	Time: 1.4060070514678955
X: 31 	Time

In [ ]:
####################################################
# END OF NORMAL CODE 
####################################################
import sys
sys.exit()

In [ ]:
from PIL import Image
import numpy as np

def showImg_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = Image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg2_(img, z=60):
    disImg = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:]).astype('uint8'))
    disImg.show()
    
def showImg3_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = image.fromarray(np.interp(a, (0, 5000), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg4_(img, z=60):
    img2 = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:])*60, 'L')
    img2.show()

In [ ]:
showImg2_(fixedImage, 60)
showImg2_(images[0], 60)
showImg2_(images[1], 60)

In [ ]:
l = sitk.GetArrayFromImage(label)
l[l == 4] = 0
mask = sitk.GetImageFromArray(l)

showImg4_(mask, 100)


In [ ]:
sitk.ImageRegistrationMethod().SetGlobalDefaultNumberOfThreads()

In [ ]:
help(sitk.CurvatureFlow)

In [ ]:
help(sitk.ImageRegistrationMethod())


In [ ]:
import SimpleITK as sitk
import os

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# spilt dataset
trainData = data[:int(datasetLen * 0.7)]
testData = data[int(datasetLen * 0.7):]

"""Write transforms."""
from pathlib import Path


pathName = os.path.join(path, testData[0])
fixedImage = sitk.ReadImage(pathName)


for i, f in enumerate(trainData[:1]): # for test
# for i, f in enumerate(trainData):
    # print(f'Registration of {f}. {i+1}/{len(trainData)}', end='\x1b[1K\r')
    print(f'Registration of {f}. {i+1}/{len(trainData)}')
    
    pathName = os.path.join(path, f)
                  
    image = sitk.ReadImage(pathName)
    label = sitk.ReadImage(os.path.join(labelPath, f))

'''
label = sitk.ReadImage(os.path.join(labelPath, f))
# Create the mask
l = sitk.GetArrayFromImage(label)
l[l>1] = 1
mask = sitk.GetImageFromArray(l)
mask.CopyInformation(label)
'''
    
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(image)
# elastixImageFilter.AddMovingMask(mask)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap("affine"))
elastixImageFilter.Execute()

In [ ]:
# resultImage2 = elastixImageFilter.GetResultImage()
transformParameterMap = elastixImageFilter.GetTransformParameterMap()

In [ ]:
showImg_(elastixImageFilter.GetResultImage())

In [ ]:
print(transformParameterMap[0].asdict())

In [ ]:
af = sitk.AffineTransform(3)
# af.SetMatrix([float(i) for i in transformParameterMap[0].asdict()['TransformParameters']])
# af.SetTranslation([float(i) for i in transformParameterMap[0].asdict()['TransformParameters']])
test = [float(i) for i in transformParameterMap[0].asdict()['TransformParameters']]
# test2 = [[i for i in test[0:3]], [i for i in test[4:7]], [i for i in test[8:11]]]
af.SetTranslation(test[9:])
af.SetMatrix(test[0:9])
test = [float(i) for i in transformParameterMap[0].asdict()['CenterOfRotationPoint']]
af.SetCenter(test)

resample = sitk.ResampleImageFilter()
resample.SetReferenceImage(fixedImage)
resample.SetInterpolator(sitk.sitkNearestNeighbor)
resample.SetTransform(af)

testL = resample.Execute(label)

# from utils import *
# testL = resampleLabels(label, fixedImage, af)

resample = sitk.ResampleImageFilter()
resample.SetReferenceImage(fixedImage)
resample.SetInterpolator(sitk.sitkLinear)
resample.SetTransform(af)
testI = resample.Execute(image)

print(resample.GetTransform())

In [ ]:
# help(sitk.ResampleImageFilter())
print(af.GetMatrix())
print(af.GetCenter())
print(af.GetTranslation())
help(af)

In [ ]:
showImg_(elastixImageFilter.GetResultImage())
# showImg_(fixedImage)
# showImg_(testL)
showImg_(testI)
# showImg_(label)
# TODO: maybe deformation fiead????

In [ ]:
sitk.GetArrayFromImage(testL)

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameter("UseBinaryFormatForTransformationParameters", "true")
transformixImageFilter.SetTransformParameterMap(elastixImageFilter.GetTransformParameterMap())
transformixImageFilter.SetMovingImage(label)
transformixImageFilter.AddTransformParameter("Interpolator", "NearestNeighborInterpolator")
transformixImageFilter.SetTransformParameter("ResultImagePixelType", "double")
transformixImageFilter.SetTransformParameter("MovingInternalImagePixelType", "double")
transformixImageFilter.SetTransformParameter("FixedInternalImagePixelType", "double")
# transformixImageFilter.SetTransformParameter("FixedInternalImagePixelType", "sitk.sitkInt8")
# transformixImageFilter.AddTransformParameter("ResampleInterpolator", "NearestNeighborInterpolator")
lel = transformixImageFilter.Execute()
showImg_(lel)

In [ ]:
# Transform label map using the deformation field from above
resultLabel = sitk.Cast(sitk.Transformix(label, elastixImageFilter.GetTransformParameterMap()), sitk.sitkInt8)
showImg_(resultLabel)
np.unique(sitk.GetArrayFromImage(resultLabel))

In [ ]:
sitk.sitkInt8

In [ ]:
np.unique(sitk.GetArrayFromImage(lel))

In [ ]:
transformixImageFilter.PrintParameterMap()

In [ ]:
print(transformixImageFilter.GetComputeSpatialJacobian())
print(transformixImageFilter.GetComputeDeterminantOfSpatialJacobian())
print(transformixImageFilter.GetComputeDeformationField())

In [ ]:
help(sitk.Transformix)

In [ ]:
help(sitk.TransformixImageFilter())

In [ ]:
help(sitk.ElastixImageFilter())

In [ ]:
test = sitk.ElastixImageFilter()
test.AddParameterMap(transformParameterMap)

In [ ]:
sitk.PrintParameterMap(elastixImageFilter.GetParameterMap())

In [ ]:
help(sitk.ElastixImageFilter)

In [ ]:
print(transformParameterMap)

In [ ]:
showImg2_(fixedImage, 60)
showImg2_(images[0], 60)
showImg2_(resultImage, 60)

In [ ]:
showImg_(fixedImage, 60)
showImg_(images[1], 60)
showImg_(resultImage, 60)

In [ ]:
help(sitk.GetImageFromArray)

In [ ]:

labels = np.array(labels, order='C', dtype=np.uint8)

segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
                              labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage,
                                                         images, 
                              np.ones(labels.shape, dtype=np.uint8, order='C')+50, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
print(fixedImage.shape)
print(fixedImage.dtype)
print(type(fixedImage))
print(images.shape)
print(images.dtype)
print(type(images))
print(labels.shape)
print(labels.dtype)
print(type(labels))

In [ ]:
from cGen import cGen
import numpy as np

P = [3, 3, 3]
N = [5, 5, 5]
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

ii = np.array(np.random.randint(0, high=500, size=(390, 276, 129)), dtype=np.uint16, order='C')
i = np.array(np.random.randint(0, high=500, size=(1, 390, 276, 129)), dtype=np.uint16, order='C')
l = np.array(np.random.randint(0, high=3, size=(1, 390, 276, 129)), dtype=np.uint8, order='C')
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(ii,
                                                              i, 
                                                              l, 3, P, N, lassoTol=0.1, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              # xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)
                              xmin=-1, xmax=-1, ymin=60, ymax=61, zmin=60, zmax=61)

In [ ]:
np.sum(l[l>0])
l

In [ ]:
import spams
import numpy as np

A = np.zeros(shape=(20, 10), dtype=np.single, order='F')
B = np.zeros(shape=(20, 1), dtype=np.single, order='F')
# B = np.zeros(shape=(20, 1), dtype=np.single, order='F')


alpha = spams.lasso(B, A, 
               return_reg_path = False, 
               lambda1 = 0.1, 
               lambda2 = 0.,
               pos = True,
               mode = 2,
               numThreads = -1,
              )


In [ ]:
type(alpha.A[:,0])

In [ ]:
alpha.toarray()[:,0]

In [ ]:

sitk.WriteImage(sitk.GetImageFromArray(fixedImage), "fixedImage.mhd")
sitk.WriteImage(sitk.GetImageFromArray(images[0]), "movingImage.mhd")
sitk.WriteImage(sitk.GetImageFromArray(labels[0]), "movinglabel.mhd")


In [ ]:
import SimpleITK as sitk
import numpy as np

fixedImage = sitk.GetArrayFromImage(sitk.ReadImage("fixedImage.mhd"))
image = sitk.GetArrayFromImage(sitk.ReadImage("movingImage.mhd"))
label = sitk.GetArrayFromImage(sitk.ReadImage("movinglabel.mhd"))

fixedImage = np.array(fixedImage, order='C')
images = np.array([image], order='C')
labels = np.array([label], order='C')

In [ ]:
print(fixedImage.shape)
print(images.shape)
print(labels.shape)

In [ ]:
from cGen import cGen

P = [3, 3, 3]
N = [5, 5, 5]
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
# segmentationSPBM, segmentationSRC = cGen.applySPBMandSRC(fixedImage, images, 
                              labels, 3, P, N, lassoTol=0.001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)
                              # xmin=60, xmax=61, ymin=-1, ymax=-1, zmin=-1, zmax=-1)
                              # xmin=10, xmax=11, ymin=10, ymax=11, zmin=10, zmax=11)

In [ ]:
showImg_(sitk.GetImageFromArray(segmentationSPBM), 60)
showImg_(sitk.GetImageFromArray(segmentationSRC), 60)
# showImg_(sitk.GetImageFromArray(originalLabel), 60)
showImg_(sitk.GetImageFromArray(labels[0]), 60)

In [ ]:
segmentationSRC[segmentationSRC == 3]

In [ ]:

originalLabel = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(labelPath, testData[0])))
translatedSegmentationSPBM = translateToOriginal(segmentationSPBM, 
                                                 os.path.join(path, testData[0]), 
                                                 copyShape, 
                                                 offset, 
                                                 length)

print(Dice(originalLabel, translatedSegmentationSPBM, 3))

translatedSegmentationSRC = translateToOriginal(segmentationSRC, 
                                                os.path.join(path, testData[0]), 
                                                copyShape, 
                                                offset, 
                                                length)

print(Dice(originalLabel, translatedSegmentationSRC, 3))